In [26]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np
import cv2 as cv
from sklearn.model_selection import train_test_split

In [28]:
# resize slike i pretvorba u grayscale(dobijemo 2 dimenzije manje -> manje parametra, a ne radi razliku jer su slike ionako crno bijele)
def preprocessing(path, path_preproc):
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Apply thresholding to the image
    thresh = cv.threshold(gray, 45, 255, cv.THRESH_BINARY)[1]

    # Find contours in the image
    contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    # Find the contour with the maximum area
    largest_contour = max(contours, key=cv.contourArea)

    extLeft = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
    extRight = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])
    extTop = tuple(largest_contour[largest_contour[:, :, 1].argmin()][0])
    extBot = tuple(largest_contour[largest_contour[:, :, 1].argmax()][0])

    # add contour on the image
    #img_cnt = cv.drawContours(img.copy(), [largest_contour], -1, (0, 255, 255), 4)
  
    #img_pnt = cv.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
    #img_pnt = cv.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
    #img_pnt = cv.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
    #img_pnt = cv.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

    #plt.title('Step 2. Find the biggest contour')
    #plt.imshow(img_pnt)
    #plt.show()

    new_img = img[extTop[1]:extBot[1], extLeft[0]:extRight[0]].copy()

    final_img = cv.resize(
            new_img,
            (200, 200),
            interpolation=cv.INTER_CUBIC
    )

    # Save the image with the largest contour
    #plt.title('Step 2. Find the biggest contour')
    #plt.imshow(img)
    #plt.show()

    cv.imwrite(path_preproc, final_img)

In [29]:
path_no = '/content/gdrive/MyDrive/asub_dataset/original_images/no/no0.jpg'
path_yes = '/content/gdrive/MyDrive/asub_dataset/original_images/yes/y0.jpg'

path_no_preproc = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/no/n0.jpg'
path_yes_preproc = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/yes/y0.jpg'


for i in range(0, 1500):
    if i != 0:
        path_no = path_no.replace(str(i - 1), str(i))
        path_yes = path_yes.replace(str(i - 1), str(i))

        path_no_preproc = path_no_preproc.replace(str(i - 1), str(i))
        path_yes_preproc = path_yes_preproc.replace(str(i - 1), str(i))

    preprocessing(path_no, path_no_preproc)
    preprocessing(path_yes, path_yes_preproc)

In [30]:
# metoda kojom samo naprim labele kako bih mogao napravit spli i to iskoristiti da znam oznaku svake slike
def make_labels():
  labels = []

  for i in range(1500):
    labels.append(0) #zero is for no

  for i in range(1500):
    labels.append(1) #one is for yes

  return labels

In [31]:
path_no_preproc = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/no/n0.jpg'
path_yes_preproc = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/yes/y0.jpg'

In [32]:
# procitam sve slike iz klase no i poslozim ih u X, onda to isto napravim za slike koje su u klasi yes
# moraju bit po redu tako da mi labele odgovaraju koje sam gore creatao
# napravi train_test split i onda cu napravit putanje i spremit sliku u odgovarajući folder

X = []

for i in range(0, 1500):
  if i != 0:
    path_no_preproc = path_no_preproc.replace(str(i - 1), str(i))
  
  img = cv.imread(path_no_preproc)
  X.append(img)

for i in range(0, 1500):
  if i != 0:
    path_yes_preproc = path_yes_preproc.replace(str(i - 1), str(i))
  
  img = cv.imread(path_yes_preproc)
  X.append(img)
  
y = make_labels()

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=37)

In [34]:
len(X_train)

1920

In [35]:
len(y_train)

1920

In [36]:
len(X_test)

600

In [37]:
len(X_validation)

480

In [38]:
# sad ću ići kroz svaki skup kako bi mogao spremiti slike u foldere i njihove oznake u .csv file
# TRAIN

path_to_save_img = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/TRAIN/'
path_train = []

for img, label in zip(X_train, y_train):
  file_name = ''

  if label == 0:
    file_name = 'no_train' + str(i) + '.jpg'
  elif label == 1:
    file_name = 'yes_train' + str(i) + '.jpg'

  full_path = path_to_save_img + file_name
  cv.imwrite(full_path, img)

  path_train.append(full_path)
  i += 1

In [39]:
# VALIDATION

path_to_save_img = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/VALIDATION/'
path_val = []

i = 0
for img, label in zip(X_validation, y_validation):
  file_name = ''

  if label == 0:
    file_name = 'no_val' + str(i) + '.jpg'
  elif label == 1:
    file_name = 'yes_val' + str(i) + '.jpg'

  full_path = path_to_save_img + file_name
  cv.imwrite(full_path, img)

  path_val.append(full_path)
  i += 1

In [40]:
# TEST

path_to_save_img = '/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/TEST/'
path_test = []

i = 0
for img, label in zip(X_test, y_test):
  file_name = ''

  if label == 0:
    file_name = 'no_test' + str(i) + '.jpg'
  elif label == 1:
    file_name = 'yes_test' + str(i) + '.jpg'

  full_path = path_to_save_img + file_name
  cv.imwrite(full_path, img)

  path_test.append(full_path)
  i += 1

In [41]:
train_df = pd.DataFrame({'label': y_train, 'path': path_train}, columns=['label', 'path'])
val_df = pd.DataFrame({'label': y_validation, 'path': path_val}, columns=['label', 'path'])
test_df = pd.DataFrame({'label': y_test, 'path': path_test}, columns=['label', 'path'])

In [42]:
train_df.to_csv(r'/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/train.csv', index = None, header=True)
val_df.to_csv(r'/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/validation.csv', index = None, header=True) 
test_df.to_csv(r'/content/gdrive/MyDrive/asub_dataset/preprocessed_newest/test.csv', index = None, header=True) 